# Apply Operations to Maximize Score
You are given an array nums of n positive integers and an integer k.

Initially, you start with a score of 1. You have to maximize your score by applying the following operation at most k times:

Choose any non-empty subarray nums[l, ..., r] that you haven't chosen previously.
Choose an element x of nums[l, ..., r] with the highest prime score. If multiple such elements exist, choose the one with the smallest index.
Multiply your score by x.
Here, nums[l, ..., r] denotes the subarray of nums starting at index l and ending at the index r, both ends being inclusive.

The prime score of an integer x is equal to the number of distinct prime factors of x. For example, the prime score of 300 is 3 since 300 = 2 * 2 * 3 * 5 * 5.

Return the maximum possible score after applying at most k operations.

Since the answer may be large, return it modulo 109 + 7.

In [ ]:
MOD = 10**9 + 7

class Solution:
    def maximumScore(self, nums: List[int], k: int) -> int:
        # 내부 함수로 prime_score 정의 (각 숫자의 서로 다른 소인수 개수를 반환)
        def prime_score(x: int) -> int:
            score = 0
            if x % 2 == 0:
                score += 1
                while x % 2 == 0:
                    x //= 2
            factor = 3
            while factor * factor <= x:
                if x % factor == 0:
                    score += 1
                    while x % factor == 0:
                        x //= factor
                factor += 2
            if x > 1:
                score += 1
            return score

        n = len(nums)
        # 1. 각 원소의 prime score 계산
        scores = [prime_score(x) for x in nums]

        # 2. 각 원소가 후보로 선택될 수 있는 서브배열 개수 계산 (monotonic stack 사용)
        left = [0] * n
        stack = []
        for i in range(n):
            # 왼쪽에서 확장 가능한 범위를 구함 (현재 원소보다 작은 값들은 팝)
            while stack and scores[stack[-1]] < scores[i]:
                stack.pop()
            left[i] = i - stack[-1] if stack else i + 1
            stack.append(i)
        
        right = [0] * n
        stack = []
        for i in range(n - 1, -1, -1):
            # 오른쪽에서는 현재 원소보다 작거나 같은 값들을 팝 (같은 경우, 왼쪽에 있는 값이 우선)
            while stack and scores[stack[-1]] <= scores[i]:
                stack.pop()
            right[i] = stack[-1] - i if stack else n - i
            stack.append(i)
        
        counts = [left[i] * right[i] for i in range(n)]
        
        # 3. (원소 값, 선택 가능한 횟수) 쌍을 내림차순 정렬한 후 그리디하게 k번 연산 할당
        pairs = [(nums[i], counts[i]) for i in range(n)]
        pairs.sort(key=lambda x: x[0], reverse=True)
        
        result = 1
        ops_left = k
        for val, cnt in pairs:
            if ops_left <= 0:
                break
            take = min(cnt, ops_left)
            result = (result * pow(val, take, MOD)) % MOD
            ops_left -= take
        
        return result
